In [7]:
!pip install tensorflow
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00


In [8]:
!nvidia-smi

Fri Jun 16 14:48:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

!pip install kaggle
!mkdir -p ~/.kaggle
!cp "/content/drive/My Drive/kaggle.json" ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat '/content/drive/My Drive/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [15]:
import json
import pandas as pd

with open('/content/sample_data/intents.json',
          'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df

,tag,patterns,responses,next-patterns
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...","[Hello there. How was your day?, Hi there. How...","[I'm having a bad day, I'm feeling so-so, I'm ..."
1,morning,"[Good morning, Morning!, Rise and shine!]",[Good morning! I hope you had a good night's s...,"[I'm having a bad day, I'm feeling so-so, I'm ..."
2,afternoon,"[Good afternoon, Afternoon!, Hey there!]",[Good afternoon! How is your day going? How wa...,"[I'm having a bad day, I'm feeling so-so, I'm ..."
3,evening,"[Good evening, Evening!, Hello!]",[Good evening! How has your day been? Anything...,"[I'm having a bad day, I'm feeling so-so, I'm ..."
4,night,"[Good night, Night!, Sweet dreams]",[Good night! Get some proper sleep. How was yo...,"[I'm having a bad day, I'm feeling so-so, I'm ..."
...,...,...,...,...
79,done,"[That's all., I don't have anything more to sa...","[I heard you and noted it all. See you later.,...","[Goodbye, Thanks]"
80,user-meditation,[i did what you said and i feel alot better. t...,[Your welcome. Remember: Always focus on what'...,"[Goodbye, Thanks]"
81,soothebot-useful,[thank you very much again. i'll continue prac...,[I'm glad you found this useful. Is there some...,"[Goodbye, Thanks]"
82,goodbye,"[Bye, See you later, Goodbye, Au revoir, Sayon...","[See you later., Have a nice day., Bye! Come b...",NaN


# Reading Data

Each tag contain multiple questions & answers so i want to sprate them

In [16]:
dic = {"tag": [], "patterns": [], "responses": []}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

In [17]:
df = pd.DataFrame.from_dict(dic)

df.head()

,tag,patterns,responses
0,greeting,Hi,"[Hello there. How was your day?, Hi there. How..."
1,greeting,Hey,"[Hello there. How was your day?, Hi there. How..."
2,greeting,Is anyone there?,"[Hello there. How was your day?, Hi there. How..."
3,greeting,Hi there,"[Hello there. How was your day?, Hi there. How..."
4,greeting,Hello,"[Hello there. How was your day?, Hi there. How..."


# Data Preprocessing

In [18]:
import numpy as np

In [19]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)


ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

number of unique words =  338
X shape =  (275, 18)
y shape =  (275,)
num of classes =  84


Now i am going to build and train model


In [20]:
import tensorflow
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Input(shape=(X.shape[1])),
    keras.layers.Embedding(input_dim=vacab_size+1,
                           output_dim=100, mask_zero=True),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LayerNormalization(),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LayerNormalization(),
    keras.layers.LSTM(64),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.LayerNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.LayerNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(len(np.unique(y)), activation="softmax")
])

model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           33900     
                                                                 
 lstm (LSTM)                 (None, 18, 64)            42240     
                                                                 
 layer_normalization (LayerN  (None, 18, 64)           128       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 18, 64)            33024     
                                                                 
 layer_normalization_1 (Laye  (None, 18, 64)           128       
 rNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 64)                3

In [21]:
history = model.fit(
    x=X,
    y=y,
    batch_size=10,
    callbacks=[tensorflow.keras.callbacks.EarlyStopping(
        monitor='accuracy', patience=3)],
    epochs=100
)

Epoch 1/100
28/28 [==============================] - 26s 166ms/step - loss: 4.8434 - accuracy: 0.0327
Epoch 2/100
28/28 [==============================] - 2s 63ms/step - loss: 2.9226 - accuracy: 0.2691
Epoch 3/100
28/28 [==============================] - 2s 63ms/step - loss: 2.0646 - accuracy: 0.4618
Epoch 4/100
28/28 [==============================] - 1s 41ms/step - loss: 1.5549 - accuracy: 0.5782
Epoch 5/100
28/28 [==============================] - 1s 28ms/step - loss: 1.0899 - accuracy: 0.7273
Epoch 6/100
28/28 [==============================] - 1s 34ms/step - loss: 0.8247 - accuracy: 0.7964
Epoch 7/100
28/28 [==============================] - 1s 28ms/step - loss: 0.6144 - accuracy: 0.8618
Epoch 8/100
28/28 [==============================] - 2s 60ms/step - loss: 0.4599 - accuracy: 0.8836
Epoch 9/100
28/28 [==============================] - 2s 61ms/step - loss: 0.4010 - accuracy: 0.9018
Epoch 10/100
28/28 [==============================] - 1s 49ms/step - loss: 0.2788 - accuracy: 0.93

In [22]:
import re
import random


def model_response(query):
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', query)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)

    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.ravel(x_test)

    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test, verbose=0)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['tag'] == tag]['responses'].values[0]

    # print("You : {}".format(query))
    print("SootheBot : {}".format(random.choice(responses)))


In [23]:
while True:
    input_text = input("You      : ")
    if input_text == 'exit':
        break
    answer = model_response(input_text)

You      : Hi!
SootheBot : Hello there. How was your day?
You      : I'm feeling so-so
SootheBot : Alright. Anything interesting you'd like to share?
You      : I'm having a bad day
SootheBot : It's tough when things don't go well. Do you want to share what's bothering you?
You      : Everything is going wrong
SootheBot : Take your time and let me know if there's anything specific you'd like to talk about.
You      : I feel sad
SootheBot : I'm sorry to hear that. I'm here for you. Talking about it might help. So, tell me why do you think you're feeling this way?
You      : I'm feeling lonely
SootheBot : I understand. Sometimes we all have bad days. Feel free to vent if you need to.
You      : I'm so stressed out
SootheBot : Sometimes a calm day can be nice. Is there anything on your mind?
You      : I feel stuck
SootheBot : Take a deep breath and gather your thoughts. Go take a walk if possible. Stay hydrated
You      : okay
SootheBot : Can you elaborate on that?
You      : yes I can
S

In [24]:
# save the model
model.save('model.h5')